# Lab 7: Optical Flow (40 Pts)

In this lab, we are going to implement the optical flow algorithm we discussed in class. Optical flow is a very useful algorithm for a wide variety of robotics tasks, including state estimation, tracking objects, and dynamically avoiding obstacles. It's also very straightforward to implement. 

Before we begin, you need to install FFMPEG to run this assignment. To do so, run `sudo apt install ffmpeg` (Linux) or `brew install ffmpeg` (Mac) in your terminal. Instructions for doing so may be found in previous labs (e.g. Lab 2). You may also need to install opencv using the following command in Anaconda Prompt (be sure the MAE345 environment is activated): `pip install opencv-python`

Also, you should be aware that a couple of the cells in this notebook render videos. While designing the assignment, the instructors found that Jupyter continues to store previous videos in memory if you rerun the cell multiple times. As a result, your computer may run out of RAM if you run the cell too many times. If this does occur, you can kill the Jupyter process in the terminal by pulling up the terminal window running Jupyter and hitting `Ctrl+C`.

In this lab, you will record a video with your crazyflie and run optical flow on this video.

Let's start by testing the camera module.

**Please note: You should work with your team to setup the camera/receiver and record the video, but you must complete the optical flow portion of the assignment individually. So, every team member should submit their own completed notebook along with the two video files produced.**

### Instructions for adding camera module for crazyflie

Remove the battery retainer, the optical flow module, and the pins attached to your board. The camera module will act as the battery holder and also keep the optical flow module attached. **Carefully** push the camera module where the battery retainer was until the pins stick out of the botton of the main body, ensure that it is facing foward on the drone and that there is enough room for the battery to slip between the main board and the camera module. Replace the optical flow board on these pins and position it as it was before. A picture is provided in `Lab7-photos`.

Next, we can use cv2 to recieve the video from the camera and view it on this computer. Begin by pluggin your receiver into this computer and then run the following cell. If the cell reports an error, ensure that opencv-python is properly installed (see above). After installing, restart the kernel.

A window which is probably showing static should appear. Next, we need to tune the receiver and camera module so they are on the same channel. The two buttons on the receiver change the received frequency down (left button) or up (right button) by 2 MHz. If you hold down the up button for a few seconds, the receiver will scan through all possibly frequencies, produce a histogram of the signal vs. frequency, and set the receiver to the frequency with the highest signal. Please note that to use this feature, you must ensure there are no other camera modudules nearby.

Channels on the camera are identified by a `BAND` (blue LED, 1 on the left and 5 on the right) and `CH` (red LED, 1 on the left and 8 on the right), e.g. `BAND1 and CH1` for the two leftmost blue and red LEDs. You can change the `CH` number by pressing the button on the camera and the `BAND` by pressing the button for at least 2 seconds and then releasing. See the chart below for the list of frequencies corresponding to a particular BAND and CH frequency in MHz.

| Channel | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Band 1   | 5865 | 5845 | 5825 | 5805 | 5785 | 5765 | 5745 | 5725 | 
| Band 2   | 5733 | 5752 | 5771 | 5790 | 5809 | 5828 | 5847 | 5866 |
| Band 3   | 5705 | 5685 | 5665 | 5665 | 5885 | 5905 | 5905 | 5905 |
| Band 4   | 5740 | 5760 | 5780 | 5800 | 5820 | 5840 | 5860 | 5880 | 
| Band 5   | 5658 | 5695 | 5732 | 5769 | 5806 | 5843 | 5880 | 5917 |

You will need to set the camera and receiver to operate on the same channel. We want to avoid intereference in the lab spaces when testing on the drone. The receiver has a large enough range that any CH and BAND can be used on the camera and the receiver will be able to pick up on the signal. However, the camera broadcasts strongest in the first 2 BANDS. Please set your camera `CH` to be the remainder of your group number when divided by 8. (i.e. if you're group number 15, you should set the `CH` 7) on Band 1 for groups 1-8, Band 2 for groups 9-16, and Band 3 for groups 17 on. This should avoid most interference but if there are issues in the lab space, you can pick another channel to tune to.

Once you are done tuning, you can press 'q' to kill the window. Note that you may have to restart the kernel by going to the 'Kernel' and clicking 'Restart' as the cv2 window sometimes causes issues with jupyter-notebook.


In [ ]:
import cv2

# This may open your webcam instead of the CrazyFlie camera! If so, try
# a different small, positive integer, e.g. 1, 2, 3.
camera = 0
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()


    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

### Instructions for recording video

Next, you will record a video with the camera module so you can perform optical flow on your very own video. The following cell will connect to the camera module on the crazyflie and record a 5-second video. The cell will tell you when the video recording starts and ends. You must manually move the crazyflie during the recording. **Be sure to include a variety of movements, including: tilting up and down, and moving forwards. We have provided a video called "optical_flow_test.avi" in the github repository that demonstrates the kinds of movements you should make.** 

The result is a video titled `video_crazyflie.avi`.

In [ ]:
# Currently 

import cv2
import time

# Remember to change this to whichever small int corresponds to the receiver from the previous code segment.
cap = cv2.VideoCapture(camera)  

fps = 20
dt = 1 / fps
image_size = (640, 480)
file_name = 'video_crazyflie.avi'

video = cv2.VideoWriter(file_name,cv2.VideoWriter_fourcc('M','J','P','G'), fps, image_size)

print('Recording video!')
for frame in range(1, fps * 5):  # record for 5 seconds
    # Capture frame-by-frame
    ret, frame = cap.read()
    video.write(frame)
    time.sleep(1 / fps)

print('Recording finished!')
# Release the capture
cap.release()
video.release()
cv2.destroyAllWindows()

Next, let's load in your video and visualize it. The following cell includes helper functions for visualization. **Do not edit this cell!**

In [ ]:
import numpy as np
import cv2
import time
from IPython.display import HTML, Image

from matplotlib import pyplot as plt
from matplotlib import animation
from typing import List, Tuple

def load_video(name):
    frames = []
    cap = cv2.VideoCapture(name)
    fps = cap.get(cv2.CAP_PROP_FPS)
    while cap.isOpened():
        ret, frame = cap.read()

        if ret == False:
            break
        else:
            frames.append(frame)

    cap.release()

    I = np.zeros([image_size[1], image_size[0], len(frames)])
    for t in range(len(frames)):
        I[:, :, t] = cv2.cvtColor(cv2.resize(frames[t], image_size), cv2.COLOR_BGR2GRAY) / 255
        
    return I, fps
    
def create_gif(I, frame_rate=20):
    fig, ax = plt.subplots()
    im = ax.imshow(I[:, :, 0], cmap='gray')

    def init():
        return [im]

    def animate(i):
        ax.clear()
        im = ax.imshow(I[:, :, i], cmap='gray')

        return [im]

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=I.shape[2], interval=100, blit=True)

    anim.save('./anim.gif', writer='pillow', fps=frame_rate)
    Image(url='./anim.gif')


The variable `I` contains a grayscale video represented as a numpy array. How videos are represented in numpy is as a three dimensional array. If you want to access the pixel at $(x, y)$ at time $t$, it is stored at `I[y, x, t]`. Moreover, all values in `I` are between 0 (black) and 1 (white).

The next cell creates a `.gif` file of the video so we can take a look at the video in motion. If the video isn't playing in jupyter-notebook, it is also saved as a `.gif` in the same directory as this notebook. Please ensure that the `.gif` file plays correctly on your computer (for Mac/Linux, the file may appear as a sequence of images). **Again, do not edit this cell!**

In [ ]:
%matplotlib inline 

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
print('Loading video!')
I, frame_rate = load_video(file_name)

print('Making GIF!')
create_gif(I, frame_rate)
print('GIF made!')

Now, we can start processing the images. Your job below is to implement the optical flow algorithm discussed in class. For a single point, $(x, y)$, the window size $w$ is the side length of the box around the pixel. That is define $h = \mathrm{floor}(w / 2)$, then the window is the set of pixels $\{(x', y')\ |\ x - h \leq x' \leq x + h, y - h\leq 'y \leq y + h\}$.

You'll also need to take a number of image derivatives. [There are a lot of ways](https://en.wikipedia.org/wiki/Image_derivatives) one can go about taking them. We'll keep it simple, however, and say that the spatial derivatives $I_x, I_y$ at $(x, y)$ can be computed by subtracting the value at $(x, y)$ from $(x + 1, y + 1)$. Please be sure to note the format of the images (written in the docstring of the function template, below), to make sure that you assign $I_x$ and $I_y$ correctly! For the temporal derivative $I_t$, you can subtract the previous image from the current one.

Using the algorithm as described so far, we'll run into trouble around the border of the image, as there are not enough pixels to create a window or potentially even take a derivative. To handle these cases, we will do the following:

1. Take the image derivatives first. Throw out all pixels for which they are not defined.

2. Create as large a window as possible around the target pixel $(x, y)$. If there are not enough pixels for a complete window, use all the pixels available in the region.

Also, in class, it was discussed that there are cases where the matrix:

$$A^{\mathrm{T}}A = \begin{bmatrix}\sum I_x I_x & \sum I_x I_y\\ \sum I_y I_x & \sum I_y I_y\end{bmatrix}$$

is not invertible. In practice, the determinant of $A^{\mathrm{T}}A$ is unlikely to be exactly zero due to the numerical approximations the computer makes. It may, however, be very small. Since we are solving for two unknowns $u, v$, we need the rank of this matrix to be at least 2. Therefore, we need at least two of the eigenvalues of $A^{\mathrm{T}}A$ to be of a reasonable size. After computing $A$, you should only solve the least-squares problem for $u, v$ if at least two eigenvalues are larger than $0.001$. Otherwise just set $u, v$ to be zero for the point at which you are evaluating the optical flow. You can compute the eigenvalues with Numpy's [eigvals](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigvals.html) function.

Be sure you are using the correct coordinates when accessing the image. It is a common pitfall.

In [ ]:
def optical_flow(prev_image: np.ndarray, image: np.ndarray, pts: np.ndarray, win_size: int) -> np.ndarray:
    """
    Compute the optical flow between two images using the Lucas-Kanade algorithm.
    
    @param prev_image: The image at time t - 1. Images are represented as arrays of the form I[y, x, t]
                       and are grayscale.
    @param image: The image at time t.
    @param pts: An n-by-2 array of points at which to evaluate the optical flow. Each point is
                 of the form [x, y].
    @param win_size: The size of the window used to compute the flow.
    
    @return: An n-by-2 array whose ith row contains the optical flow at the ith row in pts.
    """
    pass

Now lets apply it to the image. This may take some time to run. In practice, optical flow can be computed extremely quickly, but our code is slow for pedagogical purposes. If you want to just test your code quickly, change the value of `horizon` to some small value like 10. Be sure to change it back before submission, though. You can also play with the `window_size` variable to see how it impacts the computed flow. Please submit your notebook with `window_size = 30` (the default).

In [ ]:
window_size = 30

def optical_flow_video(I):
    horizon = I.shape[2]

    # Compute the optical flow at each point.
    flow = np.zeros((I.shape[0] - 1, I.shape[1] - 1, I.shape[2] - 1, 2))

    for t in range(1, horizon):
        if t % 10 == 0:
            print('Completed {0} frames.'.format(t))

        for j in range(0, flow.shape[0], 25):
            for i in range(0, flow.shape[1], 25):
                flow[j, i, t - 1] = optical_flow(I[:, :, t - 1], I[:, :, t], np.array([[i, j]]).astype(float), window_size)

    mean_flow = np.zeros((flow.shape[2], 2))

    # Compute the mean flow at each time step.
    for t in range(mean_flow.shape[0]):
        mean_flow[t, 0] = flow[:, :, t, 0].mean()
        mean_flow[t, 1] = flow[:, :, t, 1].mean()

    plt.plot(np.arange(0, mean_flow.shape[0])*dt, mean_flow[:, 0], label='X Mean')
    plt.plot(np.arange(0, mean_flow.shape[0])*dt, mean_flow[:, 1], label='Y Mean')
    plt.legend()
    return flow

The following cell runs the optical flow defined above.

In [ ]:
flow = optical_flow_video(I)

Finally, we'll plot some of the optical flow values on top of the video itself. The next cell includes helper functions for visualizing the result from optical flow. **Do not edit this cell!**

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def visualize_optical_flow(flow, I, frame_rate):
    horizon = I.shape[2]

    fig, ax = plt.subplots()

    cI = np.zeros((I.shape[0], I.shape[1], 3, I.shape[2]))

    for t in range(I.shape[2]):
        cI[:, :, :, t] = cv2.cvtColor((I[:, :, t] * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB).astype(float) / 255

    im = ax.imshow(cI[:, :, :, 0])

    num_arrows = len(range(0, flow.shape[0], 25)) * len(range(0, flow.shape[1], 25))

    cx = np.zeros(num_arrows)
    cy = np.zeros(num_arrows)
    u = np.zeros(num_arrows)
    v = np.zeros(num_arrows)

    k = 0

    for j in range(0, flow.shape[0], 25):
        for i in range(0, flow.shape[1], 25):
            cy[k] = j
            cx[k] = i
            u[k] = 0.1
            v[k] = 0.1
            k += 1   
    
    quiv = ax.quiver(cx, cy, u, v, width=.01)

    def init():
        return [im, quiv]

    def animate(t, window_size):
        boxes = []
        y_windows = int(I.shape[0] / window_size)
        x_windows = int(I.shape[1] / window_size)

        cx = np.zeros(num_arrows)
        cy = np.zeros(num_arrows)
        u = np.zeros(num_arrows)
        v = np.zeros(num_arrows)

        k = 0

        for j in range(0, flow.shape[0], 25):
            for i in range(0, flow.shape[1], 25):
                cy[k] = j
                cx[k] = i

                if np.linalg.norm(flow[j, i, t, :]) > 0.0001:
                    u[k] = 0.1 * flow[j, i, t, 0] / np.linalg.norm(flow[j, i, t, :])
                    v[k] = -0.1 * flow[j, i, t, 1] / np.linalg.norm(flow[j, i, t, :])
                else:
                    u[k] = 0
                    v[k] = 0
                k += 1
        
        ax.clear()
        im = ax.imshow(cI[:, :, :, t])

        colors = np.arctan2(u, v)
        norm = Normalize()
        colormap = cm.hsv
        
        quiv = ax.quiver(cx, cy, u, v, color=colormap(norm(colors)), width=.01)

        return [im, quiv]

    anim = animation.FuncAnimation(fig, lambda t: animate(t, window_size), init_func=init, frames=(horizon - 2), interval=100, blit=True)

    anim.save('./anim2.gif', writer='pillow', fps=frame_rate)
    Image(url='./anim2.gif')

The following cell will produce another `.gif` file which has the result from optical flow overlaid on top of your video. Again, if the video isn't playing in jupyter-notebook, it is also saved as a `.gif` in the same directory as this notebook. Please ensure that the `.gif` file plays correctly on your computer.

If you implemented things correctly, there should be a few trends you can observe in the video:
1. Arrows should generally be pointing down when the camera rotates up and pointing up when the camera rotates down.

2. When the camera moves forward (what is known as a "dolly in" motion), the arrows should generally be pointing radially outward (for example, arrows along the bottom of the image should be pointed downward). When the camera moves back ("dolly out"), the arrows should generally move radially inward.

3. Regions of the image without texture may have random arrows, as there are no trackable flow features in those regions.

These phenomena may not appear perfectly. Since the video was filmed by hand, there is some amount of shakiness that contributes to the optical flow. You can also play around with the window size to see whether or not there is improvement.

In [ ]:
visualize_optical_flow(flow, I, frame_rate)

# Submission Instructions

As usual, you should submit your completed notebook, `anim.gif`, and `anim2.gif` to HW7: Coding on Gradescope.

**Be sure to submit _both_ video files with your notebook!**